https://www.kaggle.com/datasets/theovall/phonereviews

датасет с отзывами

In [ ]:
!pip install pymorphy2
!pip install sklearn

  Using cached sklearn-0.0.post9.tar.gz (3.6 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for sklearn: filename=sklearn-0.0.post9-py3-none-any.whl size=2952 sha256=b7e5630f1e235778a7e363cb3be222282197e613af519449f21ba7cc58f9c36f
  Stored in directory: /root/.cache/pip/wheels/33/a3/d2/092b519e9522b4c91608b7dcec0dd9051fa1bff4c45f4502d1
Successfully built sklearn


In [ ]:
import csv
import pandas as pd
import nltk
import nltk
nltk.download('punkt')
from nltk.tokenize import wordpunct_tokenize
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk.corpus
from collections import Counter
import string
from pymorphy2 import MorphAnalyzer
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


1. Работа с данными

In [ ]:
reviews = pd.read_csv('/data.csv', delimiter=",")

In [ ]:
reviews.head()

,Review,Rating
0,3D Touch просто восхитительная вещь! Заряд дер...,5
1,"Отключается при температуре близкой к нулю, не...",4
2,"В Apple окончательно решили не заморачиваться,...",3
3,Постарался наиболее ёмко и коротко описать все...,4
4,Достойный телефон. Пользоваться одно удовольст...,5


In [ ]:
n_reviews = reviews[(reviews['Rating'] == 1) | (reviews['Rating'] == 2)] #собираем негативные отзывы
p_reviews = reviews[(reviews['Rating'] == 4) | (reviews['Rating'] == 5)] #собираем положительные отзывы
n_reviews['Rating'] = [0 for i in range(len(n_reviews))]
p_reviews['Rating'] = [1 for i in range(len(p_reviews))]
n_reviews = n_reviews.drop_duplicates() #убираем повторяющиеся значения
p_reviews = p_reviews.drop_duplicates()
n_reviews = n_reviews.dropna() #убираем пустые значения
p_reviews = p_reviews.dropna()

In [ ]:
neg_reviews = n_reviews[:80] #берем строки из отрицательных отзывов
pos_reviews = p_reviews[:80] #берем строки из положительных отзывов
neg_reviews_test = n_reviews[80:100] #берем для теста из отрицательных
pos_reviews_test = p_reviews[80:100] #берем для теста из положительных
reviews_test = pd.concat([neg_reviews_test, pos_reviews_test]) #соберем все тестовые отзывы

2. Работа со словами

In [ ]:
freq_n_dict = Counter()
morph = MorphAnalyzer()

for n_line in neg_reviews['Review']:
    n_line = str.lower(n_line) #приводим к нижнему регистру
    n_line = n_line.translate(str.maketrans('', '', string.punctuation)) #убираем пунктуацию
    n_line = word_tokenize(n_line) #токенизируем
    for word in n_line:
      a = morph.parse(word)
      lemma = a[0].normal_form # лемматизация
      freq_n_dict[lemma] += 1 #подсчитываем количество употреблений слов

In [ ]:
freq_p_dict = Counter()
morph = MorphAnalyzer()

for p_line in pos_reviews['Review']:
    p_line = str.lower(p_line) #приводим к нижнему регистру
    p_line = p_line.translate(str.maketrans('', '', string.punctuation)) #убираем пунктуацию
    p_line = word_tokenize(p_line) #токенизируем
    for word in p_line:
      a = morph.parse(word)
      lemma = a[0].normal_form # лемматизация
      freq_p_dict[lemma] += 1 #подсчитываем количество употреблений слов

In [ ]:
morph = MorphAnalyzer()

lemmatized_test = []
for n_line in reviews_test['Review']:
    n_line = str.lower(n_line) #приводим к нижнему регистру
    n_line = n_line.translate(str.maketrans('', '', string.punctuation)) #убираем пунктуацию
    n_line = word_tokenize(n_line) #токенизируем
    for i, word in enumerate(n_line):
      a = morph.parse(word)
      lemma = a[0].normal_form
      n_line[i] = lemma
    lemmatized_test.append(" ".join(n_line))

In [ ]:
reviews_test['Lemmas'] = lemmatized_test
reviews_test.head()

,Review,Rating,Lemmas
825,Покупать не советую. Разочарование будет от по...,0,покупать не советовать разочарование быть от п...
873,"За неделю до ДР жены,\rпосле долгого использо...",0,за неделя до др жена после долгий использовани...
885,Телефоном пользуюсь почти год. Первый аппарат ...,0,телефон пользоваться почти год первый аппарат ...
888,"Итак, \rТелефоном пользуюсь скоро как будет го...",0,итак телефон пользоваться скоро как быть год с...
906,Почитал отзывы еще раз и у меня теже самые про...,0,почитать отзыв ещё раз и у я тёж самый проблем...


3. работа со словами


In [ ]:
p_filtred_reviews = []
n_filtred_reviews = []

for word in freq_p_dict.keys():
  if freq_p_dict[word] > 2 and word not in freq_n_dict.keys():
    p_filtred_reviews.append(word)

for word in freq_n_dict:
  if freq_n_dict[word] > 2 and word not in freq_p_dict.keys():
    n_filtred_reviews.append(word)

4. Создание функции и accuracy

In [ ]:
def tonalizator(rate):  #функция для определения положительный или отрицательный отзыв
    positive_ton = 0
    negative_ton = 0
    words = word_tokenize(rate)
    for word in words:
      if word in p_filtred_reviews:
          positive_ton += 1
      elif word in n_filtred_reviews:
          negative_ton += 1
    if positive_ton >= negative_ton:
        return(1)
    else:
        return(0)

In [ ]:
quality = []

for text in reviews_test['Lemmas']:
    quality.append(tonalizator(text))

truth = reviews_test['Rating'].tolist()
score = accuracy_score(quality, truth)

print('Accuracy: ', accuracy_score(quality, truth))

Accuracy:  0.65


**Варианты улучшения:**

1. Возможно стоит попытаться использовать word2vec. Таким образом мы посмотрим на семантическую близость слов и улучшим алгоритм.

2. Возможно стоит смотреть не только на слова в отзывах, но и на их контекст, так как он бывает разным.

3. Увеличить объем данных, чтобы было точнее.

тут будут попытки парсинга, который я, видимо, не успею

In [ ]:
!pip install fake_useragent

In [ ]:
import re
import requests
from pprint import pprint

In [ ]:
session = requests.session()

from fake_useragent import UserAgent
ua = UserAgent(verify_ssl = False)

TypeError: ignored